In [1]:
import numpy as np
import pandas as pd
import xlsxwriter

In [3]:
dbT= pd.read_csv("../tables/Traffic to Contract History.txt", header = None)
exdbT= pd.read_excel("../DataFiles/Traffic to Contract History.xlsx")
dbCH= pd.read_csv("../FinalDatasets/ContractsHistory.csv")
dbCO= pd.read_csv("../FinalDatasets/Contracts.csv")
SU = pd.read_csv("../FinalDatasets/SurveyUnits.csv")
SDD = pd.read_csv("../tables/Survey Distress Details.txt", header = None)
exSDD = pd.read_excel("../DataFiles/Survey Distress Details.xlsx")
# SDD = pd.read_excel("../DataFiles/PSCdata.xlsx")

In [4]:
dbT.columns = exdbT.columns
SDD.columns = exSDD.columns

In [5]:
#dbT = dbT.drop(['Single Trucks', 'Double Trucks', 'Peak Hour Percentage', 'Peak Hour Directional Split'], axis =1)
dbT = dbT[['Traffic Year', 'Traffic Year ESALs _One Direction_', 'Contract History ID']]
dbT.columns = ['Traffic Year', 'ESALs', 'Contract History ID']

In [6]:
dbCH = dbCH[['SR','Begin ARM', 'End ARM', 'Region', 'ID', 'Construction Year', 'Major Rehab Year', 'Segment Lane Miles', 
            'ESALs Since Major Rehab', 'Total Thickness', 'Total Surface Thickness', 'Listed Surface Type', 'Pavement Type',
            'Construction Pavement Type', 'Major Rehab Type', 'Prior Major Rehab Year', 'Prior Major Rehab Type', 'UID']]

In [7]:
dbT.head()

,Traffic Year,ESALs,Contract History ID
0,2013,119738,7fbae59b-6ee6-4419-acb0-1db37aacd5f4
1,2013,91031,39483404-7908-4e89-ade2-bec2391a60b6
2,2013,91031,520f0233-9842-4d00-b92e-a10522a50102
3,2013,74348,f042adcf-fe53-4442-be3c-66080c43e0e7
4,2013,79442,5a9fd6df-7564-4251-a8ee-69f18e1f6e70


# Merging Contracts History Dataset to Traffic Loading (Y2013)

In [9]:
print len(dbT)
print len(np.unique(np.array(dbT.loc[:, 'Contract History ID'])))
print len(dbCH)

21892
21892
19253


In [10]:
T_CH = pd.merge(dbT, dbCH, how = 'left',left_on = ['Contract History ID'], right_on = ['ID'])
tch = T_CH.dropna(subset =['UID'])

In [11]:
print len(T_CH)
print len(dbT) == len(T_CH) # If false=OK, as there might be more than one contract for a given segment.
print len(tch)

23033
False
19032


# Contracts Database

In [12]:
dbC = dbCO[['SR', 'Begin ARM', 'End ARM', 'ID','Construction End Date','Substantial Completion Date','Contract Type Description', 'Surface Type Code', 'Surface Type Description', 'Surface Thickness',
          'Treat Base Type Code', 'Treat Base Type Description', 'Treat Base Thickness', 'UTreat Base Thickness', 'UID']]

In [24]:
# Function extracts the year value from the date string.
def getYear(year):
    return int(float(year.split('/')[2].split(' ')[0]))

In [14]:
# Function returns year values from a column with date format.
def returnYear(row):
    date = row['Construction End Date']
    year = getYear(date)
    return year

In [25]:
# Construction end year: Year that the specified project was ended.
dbC.loc[:, 'cYear'] = dbC.apply(returnYear, axis = 1)

/Users/Alan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/indexing.py:243: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Alan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/indexing.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [27]:
dbC[['SR', 'Begin ARM', 'End ARM', 'UID', 'Construction End Date', 'cYear']].head()

,SR,Begin ARM,End ARM,UID,Construction End Date,cYear
0,2,0.00,0.13,1,4/29/1994 0:00:00,1994
1,2,0.00,0.13,1,10/31/1997 0:00:00,1997
2,2,0.00,0.13,1,9/9/9999 0:00:00,9999
3,2,0.00,0.13,1,7/11/2008 0:00:00,2008
4,2,1.49,1.55,15,8/31/1999 0:00:00,1999


I have split up the construction date column and created a 'cYear' column which I will use to select the actions which were performed before year 2014. 

In [28]:
dbC2 = dbC[dbC.loc[:, 'cYear'] <= 2013]

In [29]:
print len(dbC)
print len(dbC2)

27224
26538


In [39]:
dbCs = dbC2

In [40]:
len(dbCs)

26538

In [41]:
print len(dbCs[dbCs.loc[:, 'UTreat Base Thickness']> 0.0])
print len(dbCs[dbCs.loc[:, 'Treat Base Thickness']> 0.0])
print len(dbCs)

4504
4761
26538


From the above summary, it can be observed that the number of segments where the treated or non-treated base layer thickness was recorded is less than 0.2 percent of the total segments in the dataset. The base thickness variable will therefore be removed for the models using records of year less than 2000. It should be noted, however, that for records of year >= 2000, there is a significant number of records with treated and untreated base-layer thicknesses so included of these variables into the models can be warranted.

In [42]:
# db = dbCs[['SR', 'Begin ARM', 'End ARM','uid', 'ID','cYear','Contract Type Description', 'Surface Type Code', 'Surface Type Description', 'Surface Thickness']]
db = dbCs #leaving all the variables in the dataset

In [43]:
print len(dbCH), len(db)

19253 26538


# Merging Contracts and merged Traffic-Contract History dataset.

In [44]:
print len(db)
print len(tch)

26538
19032


In [45]:
tch.columns

Index([u'Traffic Year', u'ESALs', u'Contract History ID', u'SR', u'Begin ARM',
       u'End ARM', u'Region', u'ID', u'Construction Year', u'Major Rehab Year',
       u'Segment Lane Miles', u'ESALs Since Major Rehab', u'Total Thickness',
       u'Total Surface Thickness', u'Listed Surface Type', u'Pavement Type',
       u'Construction Pavement Type', u'Major Rehab Type',
       u'Prior Major Rehab Year', u'Prior Major Rehab Type', u'UID'],
      dtype='object')

In [46]:
C_CH = pd.merge(db, tch, how = 'left',left_on = ['UID'], right_on = ['UID'])
CCHs = C_CH.dropna(subset = [u'Major Rehab Year'])

In [47]:
print len(CCHs), len(C_CH)

21846 30050


In [49]:
pvmtType = np.unique(np.array(CCHs[['Pavement Type']]))
print pvmtType

[nan 'ACP' 'ACP over BST (Mixed)' 'BST' 'BST over ACP (Mixed)' 'CSTS'
 'Composite' 'Grinding PCCP' 'PCCP']


In [50]:
# selected pavement type
spt = [pvmtType[1], pvmtType[2], pvmtType[3], pvmtType[4],  pvmtType[6]]
print spt

['ACP', 'ACP over BST (Mixed)', 'BST', 'BST over ACP (Mixed)', 'Composite']


In [51]:
CCHasp = CCHs[CCHs.loc[:,'Pavement Type'].isin(spt)] #selecting only asphalt pavements

Extracting the Surface types used for rehabilitation

In [53]:
CCHasp.index = np.arange(len(CCHasp))

In [54]:
MatType = np.unique(np.array(CCHasp.loc[:, 'Surface Type Code']))

In [55]:
MTdic = {}
for mt in MatType[1:]:
    MTdic[mt] = len(CCHasp[CCHasp.loc[:, 'Surface Type Code']==mt])

In [58]:
uniqMT = []
for mt in MTdic.keys():
    if MTdic[mt] > 850:
        print mt, MTdic[mt]
        uniqMT.append(mt)

BA 7175
PA 1823
SA 982
TA 1070


In [59]:
uniqMT

['BA', 'PA', 'SA', 'TA']

In [60]:
# # CCHs[~CCHs.loc[:,'Surface Type Code'].isin(uniqMT)].loc[:,'Surface Type Code']
# for i in CCHs.index:
#     if CCHs.loc[i,'Surface Type Code'] not in uniqMT:
#         CCHs.loc[i,'Surface Type Code'] = 'Other'  

I had categorized the surface type previously but later realized that the final dataset might not contain the same proportion of variables, given that I am going to consider only asphalt pavements. Categorization of this nature will be among the final steps of the data cleaning process..

Just concluded categorizing the Surface Type Code accordingly. Next step is to link the CCHs (Contract and Contract-History) dataframe to the Survey_Units dataframe using the linker dataframe uploaded below..

In [61]:
len(CCHasp[CCHasp.cYear == CCHasp.loc[:, 'Major Rehab Year']])

3100

# Merging survey units (SU) and Contracts and Contract History Datasets (CCHa)

In [34]:
# SU.columns

In [62]:
su = SU[[u'SR', u'Begin ARM', u'End ARM', 'UID', u'Is Bridge', u'Lane Surface Type', u'Lane Miles', u'Region', 
         u'Federal Functional Class', u'Int ID']]

In [63]:
su.head()

,SR,Begin ARM,End ARM,UID,Is Bridge,Lane Surface Type,Lane Miles,Region,Federal Functional Class,Int ID
0,2,312.04,312.14,3121,0,BST,0.2,Eastern,Rural Other Freeway/Expressway,1
1,2,312.04,312.14,3121,0,BST,0.2,Eastern,Rural Other Freeway/Expressway,1255
2,2,171.15,171.25,1712,0,BST,0.2,North Central,Rural Other Principal Arterial,2
3,2,269.53,269.63,2696,0,ACP,0.2,Eastern,Urban Other Principal Arterial,3
4,2,269.53,269.63,2696,0,ACP,0.2,Eastern,Urban Other Principal Arterial,2122


In [64]:
print len(CCHasp), len(su)

19110 80755


In [65]:
SCCH = pd.merge(su, CCHasp, how = 'left',left_on = ['UID'], right_on = ['UID'])
scch = SCCH.dropna(axis=0, how='any', thresh=None, subset= ['cYear'], inplace=False)

In [66]:
print len(scch)

25427


# Merging joint dataset with survey distress details

In [67]:
SDD.head()

,Year,Low Alligator Cracking,Medium Alligator Cracking,High Alligator Cracking,Low Flexible Longitudinal Cracking,Medium Flexible Longitudinal Cracking,High Flexible Longitudinal Cracking,Low Flexible Patching,Medium Flexible Patching,High Flexible Patching,...,Sealed Rigid Multi-Cracking,Sealed Alligator,Sealed Multi-Cracking,Sealed Longitudinal,Sealed Transverse,Bridge,Construction,Unratable,Intentionally Skipped,Int ID
0,2003,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54967
1,2003,2,NaN,NaN,16,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54965
2,2003,19,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54963
3,2003,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54961
4,2003,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54962


In [71]:
SD = pd.read_csv("../tables/Survey Details.txt", header = None)
exSD = pd.read_excel("../DataFiles/SurveyDetails.xlsx")
SD.columns = exSD.columns

In [72]:
sd = SD[[u'Year','Rutting', 'IRI','PSC', u'Int ID']]
sd.columns = [u'sYear','Rutting', 'IRI','PSC', u'IntID']

In [74]:
SDDs = SDD[[                                 u'Year',
                      u'Low Alligator Cracking',
                   u'Medium Alligator Cracking',
                     u'High Alligator Cracking',
          u'Low Flexible Longitudinal Cracking',
       u'Medium Flexible Longitudinal Cracking',
         u'High Flexible Longitudinal Cracking',
                       u'Low Flexible Patching',
                    u'Medium Flexible Patching',
                      u'High Flexible Patching',
            u'Low Flexible Transverse Cracking',
         u'Medium Flexible Transverse Cracking',
           u'High Flexible Transverse Cracking',
                                      u'Int ID']]

In [75]:
SDDs.columns = [                                 u'sYear',
                      u'Low Alligator Cracking',
                   u'Medium Alligator Cracking',
                     u'High Alligator Cracking',
          u'Low Flexible Longitudinal Cracking',
       u'Medium Flexible Longitudinal Cracking',
         u'High Flexible Longitudinal Cracking',
                       u'Low Flexible Patching',
                    u'Medium Flexible Patching',
                      u'High Flexible Patching',
            u'Low Flexible Transverse Cracking',
         u'Medium Flexible Transverse Cracking',
           u'High Flexible Transverse Cracking',
                                      u'IntID']

In [76]:
print len(SDDs)
print len(sd)

1845893
2646294


The following code merges the Survey Details and Survey Distress Details datasets in order to identify the years where there was measurement for the respective distresses, especially for alligator and longitudinal cracking

In [83]:
SDDs.loc[:, 'temp'] = 28

In [84]:
SDDm = pd.merge(sd, SDDs, how = 'left',left_on = ['sYear', 'IntID'], right_on = ['sYear', 'IntID'])

In [87]:
sdd = SDDm.dropna(subset = ['temp', 'PSC'])
print len(sdd)
print len(SDDm)

1500899
2646294


In [98]:
sdd = sdd.fillna(0)

In [97]:
sdd.loc[:, 'Alli'] = np.array(sdd.loc[:, 'Low Alligator Cracking']) + np.array(sdd.loc[:, 'Medium Alligator Cracking']
                    ) + np.array(sdd.loc[:, 'High Alligator Cracking'])
sdd.loc[:, 'Long'] = np.array(sdd.loc[:, 'Low Flexible Longitudinal Cracking']) + np.array(sdd.loc[:, 'Medium Flexible Longitudinal Cracking']
                    ) + np.array(sdd.loc[:, 'High Flexible Longitudinal Cracking'])

In [99]:
sdd.head()

,sYear,Rutting,IRI,PSC,IntID,Low Alligator Cracking,Medium Alligator Cracking,High Alligator Cracking,Low Flexible Longitudinal Cracking,Medium Flexible Longitudinal Cracking,High Flexible Longitudinal Cracking,Low Flexible Patching,Medium Flexible Patching,High Flexible Patching,Low Flexible Transverse Cracking,Medium Flexible Transverse Cracking,High Flexible Transverse Cracking,temp,Alli,Long
0,1981,0.000,119.091,0,1,0,316,0,0,0,686,0,0,0,0,10,0,28,316,686
1,1981,0.236,181.170,26,2,63,0,0,0,686,0,0,0,0,0,63,0,28,63,686
2,1981,0.236,85.518,87,3,0,0,0,0,0,0,0,0,0,0,10,0,28,0,0
3,1981,0.236,190.039,92,4,0,0,0,46,0,0,0,0,0,5,0,0,28,0,46
4,1981,0.236,87.418,4,5,0,316,0,0,686,0,0,0,0,0,0,0,28,316,686


Code below merges the two datasets, total and sdd.. (scct, and survey distress details)

In [100]:
allData = pd.merge(sdd, scch, how = 'left',left_on = ['IntID'], right_on = ['Int ID'])
cdata = allData.dropna(axis=0, how='any', thresh=None, subset= [u'UID'], inplace=False)

In [101]:
print len(allData)
print len(cdata), len(sdd)
print len(cdata) == len(sdd)
print len(np.unique(np.array(cdata.UID)))
print len(cdata[cdata.loc[:,'sYear']>1980])
print len(np.unique(np.array(cdata[cdata.loc[:,'sYear']>1980].loc[:, 'UID'])))

1749107
486809 1500899
False
10147
486809
10147


In [102]:
len(np.unique(np.array(cdata.loc[:, [u'SR', u'Begin ARM', u'End ARM']])))

10181

In [103]:
CDATA = cdata[[ u'sYear', u'Rutting', u'IRI', u'PSC', u'IntID', 'Alli', 'Long', u'Low Flexible Patching',
                    u'Medium Flexible Patching', u'High Flexible Patching',
            u'Low Flexible Transverse Cracking',
         u'Medium Flexible Transverse Cracking',
           u'High Flexible Transverse Cracking',
                                          u'SR',
                                   u'Begin ARM',
                                     u'End ARM',
                                         u'UID',
                                   u'Is Bridge',
                           u'Lane Surface Type',
                                  u'Lane Miles',
                                    u'Region_x',
                    u'Federal Functional Class',
                       u'Construction End Date',
                 u'Substantial Completion Date',
                   u'Contract Type Description',
                           u'Surface Type Code',
                    u'Surface Type Description',
                           u'Surface Thickness',
                        u'Treat Base Type Code',
                 u'Treat Base Type Description',
                        u'Treat Base Thickness',
                       u'UTreat Base Thickness',
                                       u'cYear',
                                u'Traffic Year',
                                       u'ESALs',
                         u'Contract History ID',
                           u'Construction Year',
                            u'Major Rehab Year',
                          u'Segment Lane Miles',
                     u'ESALs Since Major Rehab',
                             u'Total Thickness',
                     u'Total Surface Thickness',
                         u'Listed Surface Type',
                               u'Pavement Type',
                  u'Construction Pavement Type',
                            u'Major Rehab Type',
                      u'Prior Major Rehab Year',
                      u'Prior Major Rehab Type']]

In [108]:
def change2Percentage(row):
    p = row[u'Alli']
    m = row[u'Lane Miles']
    if not (p == 'nan'):
        return p/(m*5280) * 100
    else:
        return 'nan'  

In [109]:
def change2PercentageL(row):
    p = row[u'Long']
    m = row[u'Lane Miles']
    if not (p == 'nan'):
        return p/(m*5280) * 100
    else:
        return 'nan' 

In [110]:
CDATA.loc[:, 'AlliP'] = CDATA.apply(change2Percentage, axis = 1)
CDATA.loc[:, 'LongP'] = CDATA.apply(change2PercentageL, axis = 1)

In [112]:
MatType = np.unique(np.array(ds.loc[:, 'Surface Type Code']))
tds = CDATA.drop_duplicates(subset = ['UID'])

In [116]:
MTdic = {}
for mt in MatType[1:]:
    n = len(tds[tds.loc[:, 'Surface Type Code']==mt])
    if n > 500:
        MTdic[mt] = n
print len(MTdic)

3


In [117]:
MTdic

{'BA': 3962, 'PA': 879, 'SA': 521}

In [118]:
print len(tds)

10147


In [119]:
def catSurfType(row):
    st = row['Surface Type Code']
    if st in MTdic.keys():
        return st
    else:
        return 'Other'  

In [120]:
CDATA.loc[:, "STcode"] = CDATA.apply(catSurfType, axis = 1)

In [121]:
CDATA.to_csv('../FinalDatasets/workingDataset.csv')